In [2]:
%run 0.0_Dependance.ipynb

In [3]:
df = pd.read_csv("train.csv",index_col=0)
df["N_Year"]=np.round((df['N_Days']/365),2)
df.Age = np.round((df.Age)/365,1)

for i in ['Bilirubin','Cholesterol','Albumin', 'Copper','Alk_Phos',
       'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin']:
    if df[i].dtypes != "object":
        Q1 = df[i].quantile(0.25)
        Q3 = df[i].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        df["extreme_" + i] = df[i].apply(lambda x: 1 if (x < lower_bound or x > upper_bound) else 0)

#label encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
colonne_a_encoder = ["Sex","Ascites","Hepatomegaly","Spiders","Edema","Drug","Status"]	
for i in colonne_a_encoder:
    df[i] = le.fit_transform(df[i])

from sklearn.preprocessing import RobustScaler

target = df.Status
data = df.drop(columns=["Status","N_Days"])

X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.2, random_state=123)

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
df

,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,...,N_Year,extreme_Bilirubin,extreme_Cholesterol,extreme_Albumin,extreme_Copper,extreme_Alk_Phos,extreme_SGOT,extreme_Tryglicerides,extreme_Platelets,extreme_Prothrombin
id,,,,,,,,,,,,,,,,,,,,,
0,999,0,59.0,1,0,0,0,0,2.3,316.0,...,2.74,0,0,0,0,0,0,0,0,0
1,2574,1,52.7,0,0,0,0,0,0.9,364.0,...,7.05,0,0,0,0,0,0,0,0,0
2,3428,1,37.6,0,0,1,1,2,3.3,299.0,...,9.39,0,0,0,0,0,0,0,0,0
3,2576,1,50.6,0,0,0,0,0,0.6,256.0,...,7.06,0,0,0,0,0,0,0,0,0
4,788,1,45.6,0,0,1,0,0,1.1,346.0,...,2.16,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7900,1166,0,46.1,0,0,0,0,0,0.8,309.0,...,3.19,0,0,0,0,0,0,1,0,0
7901,1492,1,46.7,0,0,1,0,0,0.9,260.0,...,4.09,0,0,0,0,0,0,0,0,0
7902,1576,0,70.9,0,0,0,1,1,2.0,225.0,...,4.32,0,0,0,0,0,0,0,0,1


In [4]:
from joblib import load, dump

df_score_best_test = load("df_score_best_test.joblib")
model = XGBClassifier()
comparaison = []
for j,i in enumerate(df_score_best_test["Best_param"]):
    best_model=XGBClassifier(**i)
    best_model.fit(X_train,y_train)
    probs = best_model.predict_proba(X_test)
    logloss = log_loss(y_test, probs)
    delta = logloss-df_score_best_test["LogLoss"].values[j]
    comparaison.append(delta)

In [5]:
comparaison

[0.0030836955622207673,
 -0.00010849649058708444,
 -0.002108076261859604,
 0.0007092781853603181,
 0.0006516496611435607]